# Quality control routines for PNBOIA - ioos_qc

## Import libs

In [6]:
import numpy as np
import pandas as pd
idx = pd.IndexSlice
%load_ext autoreload
%autoreload 2
import ioos_qc
from ioos_qc.config import QcConfig
import pnboia_qc.pnboia_limits as pnboia_limits

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load PNBOIA data

In [2]:
buoys_df = pd.read_csv('../raw_data/pnboia_raw_data.csv',parse_dates=True,index_col=['buoy','Datetime'])
buoys_df.rename(columns={'humi':'rh', # variables temporarily renamed to match QCChecks limits naming
                        'dewp':'dewpt',
                        'wtmp':'sst',
                        'cvel1':'cspd1',
                        'cvel2':'cspd2',
                        'cvel3':'cspd3',
                        'wvht':'swvht',
                        'wmax':'mxwvht',
                        'dpd':'tp',
                        'mwd':'wvdir'}, inplace=True)
buoys_df.drop(columns=['lat','lon'],inplace=True)

In [5]:
config = {
    'qartod': {
        'gross_range_test': {
            'suspect_span': [1, 11],
            'fail_span': [0, 12],
        }
    }
}
c = QcConfig(config)

## Open limits for the data

In [3]:
mis_values = pnboia_limits.mis_value_axys_limits
limits = pnboia_limits.range_axys_limits
climate_limits = pnboia_limits.climate_axys_limits
stuck_limits = pnboia_limits.stuck_axys_limits
sigma_values = pnboia_limits.sigma_axys_limits
height = pnboia_limits.height
continuity_limit = pnboia_limits.continuity_axys_limits

## Create the class to start the qc tests

In [4]:
# Testing QCChecks for each buoy in the global DataFrame (buoys_df)
buoys = buoys_df.index.levels[0].unique()
for buoy in buoys:
    buoy_sel = buoys_df.loc[buoy].dropna(how='all',axis=1)
    variables = buoy_sel.columns.to_list()
    QCChecks(data=buoy_sel,
            variables=variables,
            mis_values=mis_values,
            limits=limits,
            climate_limits=climate_limits,
            stuck_limit=stuck_limits,
            sigma_values=sigma_values,
            continuity_limit=continuity_limit,
            height=height
            )
    print(f'qc object created successfully for {buoy}')
    

qc object created successfully for cf
qc object created successfully for cf2
qc object created successfully for fo
qc object created successfully for it
qc object created successfully for itaguai
qc object created successfully for itaoca
qc object created successfully for minuano
qc object created successfully for ni
qc object created successfully for ocas
qc object created successfully for po
qc object created successfully for re
qc object created successfully for rg
qc object created successfully for sa
qc object created successfully for vi


## Filtered Data Generation

In [37]:
def filter_data(buoy_name:str):
    buoy_df = buoys_df.loc[buoy_name]
    variables = buoy_df.columns.to_list()

    qc = QCChecks(data=buoy_df,
        variables=variables,
        mis_values=mis_values,
        limits=limits,
        climate_limits=climate_limits,
        stuck_limit=stuck_limits,
        sigma_values=sigma_values,
        continuity_limit=continuity_limit,
        height=height
        )

        # Missvalue test
    for parameter in mis_values.keys():
        qc.mis_value_check(parameter=parameter)

    # Range test
    for parameter in limits.keys():
        qc.range_check(parameter=parameter)

    # Climate range test
    for parameter in climate_limits.keys():
        qc.range_check_climate(parameter=parameter)

    # Comparison between swvht and mxwvht
    qc.swvht_mxwvht_check(swvht_name = 'swvht', mxwvht_name = 'mxwvht')

    # Comparison of wind speed and gust
    qc.wind_speed_gust_check(wspd_name='wspd1', gust_name='gust1')
    qc.wind_speed_gust_check(wspd_name='wspd2', gust_name='gust2')

    # Comparison of Dewpt and Atmp
    qc.dewpt_atmp_check(dewpt_name='dewpt', atmp_name='atmp')

    # Comparison of battery and pressure
    qc.bat_sensor_check(battery_name='battery', pres_name='pres')

    # Stuck sensor test
    for parameter in variables:
        if parameter != 'battery':
            print(parameter)
            qc.stuck_sensor_check(parameter=parameter)

    # Convert wind to 10 meters
    qc.convert_10_meters(wspd_name='wspd1', gust_name='gust1')
    qc.convert_10_meters(wspd_name='wspd2', gust_name='gust2')

    # Select the best anemometer
    qc.related_meas_check(parameters=['wspd1', 'wspd2', 'wdir1', 'wdir2', 'gust1', 'gust2'])

    # Time continuity test
    for parameter in sigma_values.keys():
        print(parameter)
        qc.t_continuity_check(parameter=parameter)

    # Front exception tests
    qc.front_except_check1(wdir_name='wdir', atmp_name='atmp')
    qc.front_except_check3(wspd_name='wspd', atmp_name='atmp')
    qc.front_except_check4(pres_name='pres', wspd_name='wspd')
    qc.front_except_check5(pres_name='pres')
    qc.front_except_check6(wspd_name='wspd', swvht_name='swvht')

    def filter_data_dataframe(qc_object):
        filtered_bad_data = pd.DataFrame(index=qc_object.data.index,columns=qc_object.data.columns)
        bool_array_2 = np.logical_and(qc_object.flag>0, qc_object.flag<100)
        filtered_bad_data = qc_object.data[~bool_array_2]

        return filtered_bad_data

    filtered_data = filter_data_dataframe(qc)
    filtered_data.to_csv(f'{buoy_name}_filtered.csv')

    return filtered_data

In [39]:
minuano_filtered = filter_data(buoy_name='minuano')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


In [40]:
vi_filtered = filter_data(buoy_name='vi')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


In [41]:
cf2_filtered = filter_data(buoy_name='cf2')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [42]:
itaguai_filtered = filter_data(buoy_name='itaguai')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [43]:
it_filtered = filter_data(buoy_name='it')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [44]:
ocas_filtered = filter_data(buoy_name='ocas')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [45]:
po_filtered = filter_data(buoy_name='po')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [46]:
rg_filtered = filter_data(buoy_name='rg')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [47]:
sa_filtered = filter_data(buoy_name='sa')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [48]:
cf2_filtered = filter_data(buoy_name='cf2')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


## 1. Boia Cabofrio

In [7]:
buoy = 'cf'
buoy_df = buoys_df.loc[buoy]
variables = buoy_df.columns.to_list()

qc_cf = QCChecks(data=buoy_df,
        variables=variables,
        mis_values=mis_values,
        limits=limits,
        climate_limits=climate_limits,
        stuck_limit=stuck_limits,
        sigma_values=sigma_values,
        continuity_limit=continuity_limit,
        height=height
        )

### All tests

In [8]:
# Missvalue test
for parameter in mis_values.keys():
    qc_cf.mis_value_check(parameter=parameter)

# Range test
for parameter in limits.keys():
    qc_cf.range_check(parameter=parameter)

# Climate range test
for parameter in climate_limits.keys():
    qc_cf.range_check_climate(parameter=parameter)

# Comparison between swvht and mxwvht
qc_cf.swvht_mxwvht_check(swvht_name = 'swvht', mxwvht_name = 'mxwvht')

# Comparison of wind speed and gust
qc_cf.wind_speed_gust_check(wspd_name='wspd1', gust_name='gust1')
qc_cf.wind_speed_gust_check(wspd_name='wspd2', gust_name='gust2')

# Comparison of Dewpt and Atmp
qc_cf.dewpt_atmp_check(dewpt_name='dewpt', atmp_name='atmp')

# Comparison of battery and pressure
qc_cf.bat_sensor_check(battery_name='battery', pres_name='pres')

# Stuck sensor test
for parameter in variables:
    if parameter != 'battery':
        print(parameter)
        qc_cf.stuck_sensor_check(parameter=parameter)

# Convert wind to 10 meters
qc_cf.convert_10_meters(wspd_name='wspd1', gust_name='gust1')
qc_cf.convert_10_meters(wspd_name='wspd2', gust_name='gust2')

# Select the best anemometer
qc_cf.related_meas_check(parameters=['wspd1', 'wspd2', 'wdir1', 'wdir2', 'gust1', 'gust2'])

# Time continuity test
for parameter in sigma_values.keys():
    print(parameter)
    qc_cf.t_continuity_check(parameter=parameter)

# Front exception tests
qc_cf.front_except_check1(wdir_name='wdir', atmp_name='atmp')
qc_cf.front_except_check3(wspd_name='wspd', atmp_name='atmp')
qc_cf.front_except_check4(pres_name='pres', wspd_name='wspd')
qc_cf.front_except_check5(pres_name='pres')
qc_cf.front_except_check6(wspd_name='wspd', swvht_name='swvht')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


AttributeError: 'QCChecks' object has no attribute 'frbont_except_check6'

In [11]:
qc_cf.flag.head()

,battery,wspd1,gust1,wdir1,wspd2,gust2,wdir2,atmp,rh,dewpt,...,cspd3,cdir3,swvht,mxwvht,tp,wvdir,spred,wspd,wdir,gust
Datetime,,,,,,,,,,,,,,,,,,,,,
2009-06-24 15:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009-06-24 16:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009-06-24 17:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009-06-24 18:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2009-06-24 19:00:00,0,0,0,0,0,0,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
def filter_data(qc_object):
    filtered_bad_data = pd.DataFrame(index=qc_object.data.index,columns=qc_object.data.columns)
    bool_array_2 = np.logical_and(qc_object.flag>0, qc_object.flag<100)
    filtered_bad_data = qc_object.data[~bool_array_2]

    return filtered_bad_data


In [19]:
cf_filtered = filter_data(qc_cf)
cf_filtered.to_csv('cf_filtered.csv')

## 6. Boia Niteroi

In [12]:
buoy = 'ni'
buoy_df = buoys_df.loc[buoy]
variables = buoy_df.columns.to_list()

qc_ni = QCChecks(data=buoy_df,
        variables=variables,
        mis_values=mis_values,
        limits=limits,
        climate_limits=climate_limits,
        stuck_limit=stuck_limits,
        sigma_values=sigma_values,
        continuity_limit=continuity_limit,
        height=height
        )

### All tests

In [13]:
# Missvalue test
for parameter in mis_values.keys():
    qc_ni.mis_value_check(parameter=parameter)

# Range test
for parameter in limits.keys():
    qc_ni.range_check(parameter=parameter)

# Climate range test
for parameter in climate_limits.keys():
    qc_ni.range_check_climate(parameter=parameter)

# Comparison between swvht and mxwvht
qc_ni.swvht_mxwvht_check(swvht_name = 'swvht', mxwvht_name = 'mxwvht')

# Comparison of wind speed and gust
qc_ni.wind_speed_gust_check(wspd_name='wspd1', gust_name='gust1')
qc_ni.wind_speed_gust_check(wspd_name='wspd2', gust_name='gust2')

# Comparison of Dewpt and Atmp
qc_ni.dewpt_atmp_check(dewpt_name='dewpt', atmp_name='atmp')

# Comparison of battery and pressure
qc_ni.bat_sensor_check(battery_name='battery', pres_name='pres')

# Stuck sensor test
for parameter in variables:
    if parameter != 'battery':
        print(parameter)
        qc_ni.stuck_sensor_check(parameter=parameter)

# Convert wind to 10 meters
qc_ni.convert_10_meters(wspd_name='wspd1', gust_name='gust1')
qc_ni.convert_10_meters(wspd_name='wspd2', gust_name='gust2')

# Select the best anemometer
qc_ni.related_meas_check(parameters=['wspd1', 'wspd2', 'wdir1', 'wdir2', 'gust1', 'gust2'])

# Time continuity test
for parameter in sigma_values.keys():
    print(parameter)
    qc_ni.t_continuity_check(parameter=parameter)

# Front exception tests
qc_ni.front_except_check1(wdir_name='wdir', atmp_name='atmp')
qc_ni.front_except_check3(wspd_name='wspd', atmp_name='atmp')
qc_ni.front_except_check4(pres_name='pres', wspd_name='wspd')
qc_ni.front_except_check5(pres_name='pres')
qc_ni.front_except_check6(wspd_name='wspd', swvht_name='swvht')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [21]:
ni_filtered = filter_data(qc_ni)
ni_filtered.to_csv('ni_filtered.csv')

## 10. Boia Fortaleza

In [14]:
buoy = 'fo'
buoy_df = buoys_df.loc[buoy]
variables = buoy_df.columns.to_list()

qc_fo = QCChecks(data=buoy_df,
        variables=variables,
        mis_values=mis_values,
        limits=limits,
        climate_limits=climate_limits,
        stuck_limit=stuck_limits,
        sigma_values=sigma_values,
        continuity_limit=continuity_limit,
        height=height
        )

### All tests

In [15]:
# Missvalue test
for parameter in mis_values.keys():
    qc_fo.mis_value_check(parameter=parameter)

# Range test
for parameter in limits.keys():
    qc_fo.range_check(parameter=parameter)

# Climate range test
for parameter in climate_limits.keys():
    qc_fo.range_check_climate(parameter=parameter)

# Comparison between swvht and mxwvht
qc_fo.swvht_mxwvht_check(swvht_name = 'swvht', mxwvht_name = 'mxwvht')

# Comparison of wind speed and gust
qc_fo.wind_speed_gust_check(wspd_name='wspd1', gust_name='gust1')
qc_fo.wind_speed_gust_check(wspd_name='wspd2', gust_name='gust2')

# Comparison of Dewpt and Atmp
qc_fo.dewpt_atmp_check(dewpt_name='dewpt', atmp_name='atmp')

# Comparison of battery and pressure
qc_fo.bat_sensor_check(battery_name='battery', pres_name='pres')

# Stuck sensor test
for parameter in variables:
    if parameter != 'battery':
        print(parameter)
        qc_fo.stuck_sensor_check(parameter=parameter)

# Convert wind to 10 meters
qc_fo.convert_10_meters(wspd_name='wspd1', gust_name='gust1')
qc_fo.convert_10_meters(wspd_name='wspd2', gust_name='gust2')

# Select the best anemometer
qc_fo.related_meas_check(parameters=['wspd1', 'wspd2', 'wdir1', 'wdir2', 'gust1', 'gust2'])

# Time continuity test
for parameter in sigma_values.keys():
    print(parameter)
    qc_fo.t_continuity_check(parameter=parameter)

# Front exception tests
qc_fo.front_except_check1(wdir_name='wdir', atmp_name='atmp')
qc_fo.front_except_check3(wspd_name='wspd', atmp_name='atmp')
qc_fo.front_except_check4(pres_name='pres', wspd_name='wspd')
qc_fo.front_except_check5(pres_name='pres')
qc_fo.front_except_check6(wspd_name='wspd', swvht_name='swvht')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


In [22]:
fo_filtered = filter_data(qc_fo)
fo_filtered.to_csv('fo_filtered.csv')

## 10. Boia Itaoca

In [23]:
buoy = 'itaoca'
buoy_df = buoys_df.loc[buoy]
variables = buoy_df.columns.to_list()

qc_itaoca = QCChecks(data=buoy_df,
        variables=variables,
        mis_values=mis_values,
        limits=limits,
        climate_limits=climate_limits,
        stuck_limit=stuck_limits,
        sigma_values=sigma_values,
        continuity_limit=continuity_limit,
        height=height
        )

### All tests

In [24]:
# Missvalue test
for parameter in mis_values.keys():
    qc_itaoca.mis_value_check(parameter=parameter)

# Range test
for parameter in limits.keys():
    qc_itaoca.range_check(parameter=parameter)

# Climate range test
for parameter in climate_limits.keys():
    qc_itaoca.range_check_climate(parameter=parameter)

# Comparison between swvht and mxwvht
qc_itaoca.swvht_mxwvht_check(swvht_name = 'swvht', mxwvht_name = 'mxwvht')

# Comparison of wind speed and gust
qc_itaoca.wind_speed_gust_check(wspd_name='wspd1', gust_name='gust1')
qc_itaoca.wind_speed_gust_check(wspd_name='wspd2', gust_name='gust2')

# Comparison of Dewpt and Atmp
qc_itaoca.dewpt_atmp_check(dewpt_name='dewpt', atmp_name='atmp')

# Comparison of battery and pressure
qc_itaoca.bat_sensor_check(battery_name='battery', pres_name='pres')

# Stuck sensor test
for parameter in variables:
    if parameter != 'battery':
        print(parameter)
        qc_itaoca.stuck_sensor_check(parameter=parameter)

# Convert wind to 10 meters
qc_itaoca.convert_10_meters(wspd_name='wspd1', gust_name='gust1')
qc_itaoca.convert_10_meters(wspd_name='wspd2', gust_name='gust2')

# Select the best anemometer
qc_itaoca.related_meas_check(parameters=['wspd1', 'wspd2', 'wdir1', 'wdir2', 'gust1', 'gust2'])

# Time continuity test
for parameter in sigma_values.keys():
    print(parameter)
    qc_itaoca.t_continuity_check(parameter=parameter)

# Front exception tests
qc_itaoca.front_except_check1(wdir_name='wdir', atmp_name='atmp')
qc_itaoca.front_except_check3(wspd_name='wspd', atmp_name='atmp')
qc_itaoca.front_except_check4(pres_name='pres', wspd_name='wspd')
qc_itaoca.front_except_check5(pres_name='pres')
qc_itaoca.front_except_check6(wspd_name='wspd', swvht_name='swvht')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


In [27]:
itaoca_filtered = filter_data(qc_itaoca)
itaoca_filtered.to_csv('itaoca_filtered.csv')

## 11. Boia Itajaí

In [25]:
buoy = 'it'
buoy_df = buoys_df.loc[buoy]
variables = buoy_df.columns.to_list()

qc_it = QCChecks(data=buoy_df,
        variables=variables,
        mis_values=mis_values,
        limits=limits,
        climate_limits=climate_limits,
        stuck_limit=stuck_limits,
        sigma_values=sigma_values,
        continuity_limit=continuity_limit,
        height=height
        )

### All tests

In [26]:
# Missvalue test
for parameter in mis_values.keys():
    qc_it.mis_value_check(parameter=parameter)

# Range test
for parameter in limits.keys():
    qc_it.range_check(parameter=parameter)

# Climate range test
for parameter in climate_limits.keys():
    qc_it.range_check_climate(parameter=parameter)

# Comparison between swvht and mxwvht
qc_it.swvht_mxwvht_check(swvht_name = 'swvht', mxwvht_name = 'mxwvht')

# Comparison of wind speed and gust
qc_it.wind_speed_gust_check(wspd_name='wspd1', gust_name='gust1')
qc_it.wind_speed_gust_check(wspd_name='wspd2', gust_name='gust2')

# Comparison of Dewpt and Atmp
qc_it.dewpt_atmp_check(dewpt_name='dewpt', atmp_name='atmp')

# Comparison of battery and pressure
qc_it.bat_sensor_check(battery_name='battery', pres_name='pres')

# Stuck sensor test
for parameter in variables:
    if parameter != 'battery':
        print(parameter)
        qc_it.stuck_sensor_check(parameter=parameter)

# Convert wind to 10 meters
qc_it.convert_10_meters(wspd_name='wspd1', gust_name='gust1')
qc_it.convert_10_meters(wspd_name='wspd2', gust_name='gust2')

# Select the best anemometer
qc_it.related_meas_check(parameters=['wspd1', 'wspd2', 'wdir1', 'wdir2', 'gust1', 'gust2'])

# Time continuity test
for parameter in sigma_values.keys():
    print(parameter)
    qc_it.t_continuity_check(parameter=parameter)

# Front exception tests
qc_it.front_except_check1(wdir_name='wdir', atmp_name='atmp')
qc_it.front_except_check3(wspd_name='wspd', atmp_name='atmp')
qc_it.front_except_check4(pres_name='pres', wspd_name='wspd')
qc_it.front_except_check5(pres_name='pres')
qc_it.front_except_check6(wspd_name='wspd', swvht_name='swvht')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


## 11. Boia Recife

In [28]:
buoy = 're'
buoy_df = buoys_df.loc[buoy]
variables = buoy_df.columns.to_list()

qc_re = QCChecks(data=buoy_df,
        variables=variables,
        mis_values=mis_values,
        limits=limits,
        climate_limits=climate_limits,
        stuck_limit=stuck_limits,
        sigma_values=sigma_values,
        continuity_limit=continuity_limit,
        height=height
        )

### All tests

In [29]:
# Missvalue test
for parameter in mis_values.keys():
    qc_re.mis_value_check(parameter=parameter)

# Range test
for parameter in limits.keys():
    qc_re.range_check(parameter=parameter)

# Climate range test
for parameter in climate_limits.keys():
    qc_re.range_check_climate(parameter=parameter)

# Comparison between swvht and mxwvht
qc_re.swvht_mxwvht_check(swvht_name = 'swvht', mxwvht_name = 'mxwvht')

# Comparison of wind speed and gust
qc_re.wind_speed_gust_check(wspd_name='wspd1', gust_name='gust1')
qc_re.wind_speed_gust_check(wspd_name='wspd2', gust_name='gust2')

# Comparison of Dewpt and Atmp
qc_re.dewpt_atmp_check(dewpt_name='dewpt', atmp_name='atmp')

# Comparison of battery and pressure
qc_re.bat_sensor_check(battery_name='battery', pres_name='pres')

# Stuck sensor test
for parameter in variables:
    if parameter != 'battery':
        print(parameter)
        qc_re.stuck_sensor_check(parameter=parameter)

# Convert wind to 10 meters
qc_re.convert_10_meters(wspd_name='wspd1', gust_name='gust1')
qc_re.convert_10_meters(wspd_name='wspd2', gust_name='gust2')

# Select the best anemometer
qc_re.related_meas_check(parameters=['wspd1', 'wspd2', 'wdir1', 'wdir2', 'gust1', 'gust2'])

# Time continuity test
for parameter in sigma_values.keys():
    print(parameter)
    qc_re.t_continuity_check(parameter=parameter)

# Front exception tests
qc_re.front_except_check1(wdir_name='wdir', atmp_name='atmp')
qc_re.front_except_check3(wspd_name='wspd', atmp_name='atmp')
qc_re.front_except_check4(pres_name='pres', wspd_name='wspd')
qc_re.front_except_check5(pres_name='pres')
qc_re.front_except_check6(wspd_name='wspd', swvht_name='swvht')

/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:268: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[dewpt_name] == 51), dewpt_name] = self.data[atmp_name]


wspd1
gust1
wdir1
wspd2
gust2
wdir2
atmp
rh
dewpt
pres
sst
bhead
arad
cspd1
cdir1
cspd2
cdir2
cspd3
cdir3
swvht
mxwvht
tp
wvdir
spred


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:340: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[wspd_name] == 0), wspd_name] = self.data[wspd_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:342: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.data.loc[(self.flag[gust_name] == 0), gust_name] = self.data[gust_name] * (10 / height) ** 0.11
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:361: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

swvht
rh
pres
atmp
wspd
sst


/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]
/home/remobs-dados/pnboia_qc/pnboia_qc/qc_checks.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select_value["pres_new"] = select_value_2["pres"]


In [30]:
re_filtered = filter_data(qc_re)
re_filtered.to_csv('re_filtered.csv')